<a href="https://colab.research.google.com/github/dotsnangles/from-keras-to-pytorch-and-more/blob/main/stylegan2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Training StyleGAN2 in Google CoLab

GANs can be trained with either Google Colab Free or Pro.  The Pro version is reccomended due to better GPU instances, longer runtimes, and timeouts.  Make sure that you are running this notebook with a GPU runtime.

Your training data and trained neural networks will be stored to GDRIVE.  For GANs, I lay out my GDRIVE like this:

* ./data/gan/images - RAW images I wish to train on.
* ./data/gan/datasets - Actual training datasets that I convert from the raw images.
* ./data/gan/experiments - The output from StyleGAN2, my image previews and saved network snapshots.

The drive is mounted to the following location.

```
/content/drive/MyDrive/data
```


# What Sort of GPU do you Have?

The type of GPU assigned to you by Colab will greatly affect your training time. Some sample times that I achieved with Colab are given here.  I've found that Colab Pro generally starts you with a V100, however, if you run scripts non-stop for 24hrs straight for a few days in a row, you will generally be throttled back to a P100.

* 1024x1024 - V100 - 566 sec/tick (CoLab Pro)
* 1024x1024 - P100 - 1819 sec/tick (CoLab Pro)
* 1024x1024 - T4 - 2188 sec/tick (CoLab Free)

If you use Google CoLab Pro, generally, it will not disconnect before 24 hours, even if you (but not your script) are inactive.  Free CoLab WILL disconnect a perfectly good running script if you do not interact for a few hours.  The following describes how to circumvent this issue.

* [How to prevent Google Colab from disconnecting?](https://stackoverflow.com/questions/57113226/how-to-prevent-google-colab-from-disconnecting)


In [ ]:
!nvidia-smi

Thu Dec 29 02:42:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P0    30W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Set Up New Environment

You will likely need to train for >24 hours.  Colab will disconnect you.  You must be prepared to restart training when this eventually happens.  Training is divided into ticks, every so many ticks (50 by default) your neural network is evaluated and a snapshot is saved.  When CoLab shuts down, all training after the last snapshot is lost. It might seem desirable to snapshot after each tick; however, this snapshotting process itself takes nearly an hour.  It is important to learn an optimal snapshot size for your resolution and training data.

We will mount GDRIVE so that your snapshots are saved there.  You must also place your training images in GDRIVE.

In [1]:
try:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Mounted at /content/drive
Note: using Google CoLab


You must also install NVIDIA StyleGAN2 ADA PyTorch.  We also need to downgrade PyTorch to a version that supports StyleGAN.

In [2]:
!pip install -q torch==1.8.1 torchvision==0.9.1 flickrapi ninja
!git clone https://github.com/NVlabs/stylegan2-ada-pytorch.git
!git clone https://github.com/jeffheaton/pyimgdata.git

     |████████████████████████████████| 804.1 MB 6.2 kB/s 
     |████████████████████████████████| 17.4 MB 283 kB/s 
     |████████████████████████████████| 145 kB 58.4 MB/s 
     |████████████████████████████████| 54 kB 3.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchtext 0.14.0 requires torch==1.13.0, but you have torch 1.8.1 which is incompatible.
torchaudio 0.13.0+cu116 requires torch==1.13.0, but you have torch 1.8.1 which is incompatible.
Cloning into 'stylegan2-ada-pytorch'...
remote: Enumerating objects: 128, done.
remote: Total 128 (delta 0), reused 0 (delta 0), pack-reused 128
Receiving objects: 100% (128/128), 1.12 MiB | 35.87 MiB/s, done.
Resolving deltas: 100% (57/57), done.
Cloning into 'pyimgdata'...
remote: Enumerating objects: 31, done.
remote: Counting objects: 100% (31/31), done.
remote: Compressing objects: 100% (24/24), done.

# 폴더를 생성하고 크롤링을 시작합니다.

In [ ]:
!mkdir -p /content/drive/MyDrive/data/gan/images
!mkdir -p /content/drive/MyDrive/data/gan/datasets
!mkdir -p /content/drive/MyDrive/data/gan/experiments 

-  flickr 계정을 생성하고 api key/secret을 부여받아야 합니다.
- /content/pyimgdata/config_flickr.ini 설정이 필요합니다.

In [ ]:
%cd /content/pyimgdata
!python flickr-download.py
%cd /content

/content/pyimgdata
2022-12-29 01:48:57,140 - root - INFO : Line 166 - Starting...
2022-12-29 01:49:57,247 - root - INFO : Line 147 - Update for 1: images=163; errors=0; cached=59
2022-12-29 01:50:57,162 - root - INFO : Line 147 - Update for 2: images=322; errors=0; cached=93
2022-12-29 01:51:57,346 - root - INFO : Line 147 - Update for 3: images=493; errors=0; cached=95
2022-12-29 01:52:57,225 - root - INFO : Line 147 - Update for 4: images=654; errors=0; cached=147
2022-12-29 01:53:57,470 - root - INFO : Line 147 - Update for 5: images=819; errors=0; cached=168
2022-12-29 01:54:57,493 - root - INFO : Line 147 - Update for 6: images=984; errors=0; cached=179
2022-12-29 01:55:57,414 - root - INFO : Line 147 - Update for 7: images=1,135; errors=0; cached=272
2022-12-29 01:56:57,443 - root - INFO : Line 147 - Update for 8: images=1,298; errors=0; cached=272
2022-12-29 01:57:57,494 - root - INFO : Line 147 - Update for 9: images=1,466; errors=0; cached=272
2022-12-29 01:58:57,179 - root - 

# Clean Up your Images

It is important that all images have the same dimensions and color depth.  This code can identify images that have issues.

In [ ]:
from os import listdir
from os.path import isfile, join
import os
from PIL import Image
from tqdm.notebook import tqdm

IMAGE_PATH = '/content/drive/MyDrive/data/gan/images'
files = [f for f in listdir(IMAGE_PATH) if isfile(join(IMAGE_PATH, f))]

base_size = None
for file in tqdm(files):
  file2 = os.path.join(IMAGE_PATH,file)
  img = Image.open(file2)
  sz = img.size
  if base_size and sz!=base_size:
    print(f"Inconsistant size: {file2}")
  elif img.mode!='RGB':
    print(f"Inconsistant color format: {file2}")
  else:
    base_size = sz


  0%|          | 0/3220 [00:00<?, ?it/s]

# Convert Your Images

In [ ]:
!python /content/stylegan2-ada-pytorch/dataset_tool.py --source /content/drive/MyDrive/data/gan/images --dest /content/drive/MyDrive/data/gan/dataset

100% 3220/3220 [01:09<00:00, 46.39it/s]


The following command can be used to clear out the newly created dataset.  If something goes wrong and you need to clean up your images and rerun the above command, you should delete your partially created dataset directory.

In [ ]:
#!rm -R /content/drive/MyDrive/data/gan/dataset/*

# Perform Initial Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 10
WORKERS = 2

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --outdir {EXPERIMENTS} --data {DATA} --workers {WORKERS}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/data/gan/dataset",
    "use_labels": false,
    "max_size": 3220,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 2,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "c

# Resume Training

In [ ]:
import os

# Modify these to suit your needs
EXPERIMENTS = "/content/drive/MyDrive/data/gan/experiments"
NETWORK = "network-snapshot-000120.pkl"
RESUME = os.path.join(EXPERIMENTS, "00008-dataset-auto1-resumecustom", NETWORK)
DATA = "/content/drive/MyDrive/data/gan/dataset"
SNAP = 10
WORKERS = 2

# Build the command and run it
cmd = f"/usr/bin/python3 /content/stylegan2-ada-pytorch/train.py --snap {SNAP} --resume {RESUME} --outdir {EXPERIMENTS} --data {DATA} --workers {WORKERS}"
!{cmd}


Training options:
{
  "num_gpus": 1,
  "image_snapshot_ticks": 10,
  "network_snapshot_ticks": 10,
  "metrics": [
    "fid50k_full"
  ],
  "random_seed": 0,
  "training_set_kwargs": {
    "class_name": "training.dataset.ImageFolderDataset",
    "path": "/content/drive/MyDrive/data/gan/dataset",
    "use_labels": false,
    "max_size": 3220,
    "xflip": false,
    "resolution": 256
  },
  "data_loader_kwargs": {
    "pin_memory": true,
    "num_workers": 2,
    "prefetch_factor": 2
  },
  "G_kwargs": {
    "class_name": "training.networks.Generator",
    "z_dim": 512,
    "w_dim": 512,
    "mapping_kwargs": {
      "num_layers": 2
    },
    "synthesis_kwargs": {
      "channel_base": 16384,
      "channel_max": 512,
      "num_fp16_res": 4,
      "conv_clamp": 256
    }
  },
  "D_kwargs": {
    "class_name": "training.networks.Discriminator",
    "block_kwargs": {},
    "mapping_kwargs": {},
    "epilogue_kwargs": {
      "mbstd_group_size": 4
    },
    "channel_base": 16384,
    "c